In [123]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
#import imageio
import cv2
import os
import json

Data preparation

V2_trailer_Dataset

In [20]:
# # reading the JSON data using json.load()
# file = '/home/app/src/data/shot-type-dataset/v2_full_trailer.json'
# with open(file) as train_file:
#     dict_v2= json.load(train_file)


In [21]:
# # Creating a DataFrame with the v2_full_trailer.json"

# dataset = []
# movie = []
# shot= []
# scale_label = []
# scale_val = []
# mov_label = []
# mov_val = []

# # We pass for all JSON keys an append every trailer labels different lists
# for key_movie in dict_v2:
#     for key_shot in dict_v2[key_movie]:

#         dataset.append('v2')
#         movie.append(key_movie)
#         shot.append(key_shot)
#         scale_label.append('None')
#         scale_val.append(int(0))
#         mov_label.append(dict_v2[key_movie][key_shot]['movement']['label'])
#         mov_val.append(dict_v2[key_movie][key_shot]['movement']['value'])

# # We build the dataframe with the lists data
# data = list(zip(dataset, movie, shot, scale_label, scale_val, mov_label, mov_val))  
# columns = ['dataset','movie', 'shot', 'scale_label', 'scale_val', 'move_label', 'move_val']  
        
# dataset_df_v2 = pd.DataFrame(data, columns=columns)

In [22]:
# v2_df = dataset_df_v2
# v2_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v2/'
# v2_df ['video_name'] = v2_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
# v2_df = v2_df[['video_name', 'move_label','scale_label']]
# v2_df = v2_df.rename(columns ={'scale_label': 'tag'})



In [23]:
#v2_df.to_csv('/home/app/src/data/CSV/dataset_v2.csv')

In [24]:
#v2_df= pd.read_csv('/home/app/src/data/CSV/dataset_v2.csv')
#v2_df

Preparing Trailer_v3_dataset (The complete dataset)

In [124]:
data_df = pd. read_csv('/home/app/src/data/CSV/dataset_df.csv', dtype={"shot": str})

data_df.drop(columns=['scale_val'], inplace=True)
data_df.drop(columns = ['move_val'], inplace=True)

train_path = '/home/app/src/data/shot-type-dataset/trailer_v3/train/'
data_df['video_name'] = train_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
train_df = data_df[(data_df['dataset']=='train') | (data_df['dataset']=='val') ]
train_df = train_df[['video_name', 'move_label','scale_label']]
train_df = train_df.rename(columns ={'scale_label': 'tag'})


test_path = '/home/app/src/data/shot-type-dataset/trailer_v3/test/'
data_df['video_name'] = test_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
test_df = data_df[data_df['dataset']=='test']
test_df = test_df[['video_name', 'move_label','scale_label']]
test_df = test_df.rename(columns ={'scale_label': 'tag'})

In [125]:
train_df 

,video_name,move_label,tag
0,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
1,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
2,/home/app/src/data/shot-type-dataset/trailer_v...,Static,ECS
3,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
4,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
...,...,...,...
25461,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
25462,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,MS
25463,/home/app/src/data/shot-type-dataset/trailer_v...,Static,FS
25464,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS


In [126]:
test_df

,video_name,move_label,tag
25466,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS
25467,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
25468,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
25469,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS
25470,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,FS
...,...,...,...
33648,/home/app/src/data/shot-type-dataset/trailer_v...,Static,FS
33649,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,MS
33650,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,FS
33651,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS


Creating TRAIN balanced dataset

In [127]:
train_df_bal_static = train_df[train_df.move_label == 'Static'][0:700]
train_df_bal_motion = train_df[train_df.move_label == 'Motion'][0:700]

len_push_tag = train_df.move_label[train_df.move_label == 'Push'].shape[0]
len_pull_tag = train_df.move_label[train_df.move_label == 'Pull'].shape[0]

train_df_bal_push = train_df[train_df.move_label == 'Push'][0:len_push_tag]
train_df_bal_pull = train_df[train_df.move_label == 'Pull'][0:len_pull_tag]


In [128]:
train_df_bal = pd.concat([train_df_bal_static, train_df_bal_motion, train_df_bal_pull, train_df_bal_push], axis=0)

In [129]:
train_df_bal

,video_name,move_label,tag
0,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
1,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
2,/home/app/src/data/shot-type-dataset/trailer_v...,Static,ECS
3,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
4,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
...,...,...,...
25429,/home/app/src/data/shot-type-dataset/trailer_v...,Push,FS
25436,/home/app/src/data/shot-type-dataset/trailer_v...,Push,LS
25447,/home/app/src/data/shot-type-dataset/trailer_v...,Push,MS
25449,/home/app/src/data/shot-type-dataset/trailer_v...,Push,ECS


Adding v2_trailers to Training dataset

In [31]:
# new_idx_v2 = []

# for x in range(train_df_bal.index[-1]+1,train_df_bal.index[-1]+1+len(v2_df.index )):
#     new_idx_v2.append(x)

In [32]:
#v2_df.index = new_idx_v2
#v2_df = v2_df.drop(columns='Unnamed: 0')

In [33]:
# = pd.concat([train_df_bal, v2_df], axis=0)

In [34]:
#train_df_bal.video_name[1500]

In [35]:
#train_df_bal.shape

In [36]:
#train_df_bal

Creating TEST balanced dataset

In [130]:
test_df_bal_static = test_df[test_df.move_label == 'Static'][0:150]
test_df_bal_motion = test_df[test_df.move_label == 'Motion'][0:150]

len_push_tag = test_df.move_label[test_df.move_label == 'Push'].shape[0]
len_pull_tag = test_df.move_label[test_df.move_label == 'Pull'].shape[0]

test_df_bal_push = test_df[test_df.move_label == 'Push'][0:len_push_tag]
test_df_bal_pull = test_df[test_df.move_label == 'Pull'][0:len_pull_tag]


In [131]:
test_df_bal = pd.concat([test_df_bal_static, test_df_bal_motion, test_df_bal_pull, test_df_bal_push], axis=0)

In [132]:
test_df_bal

,video_name,move_label,tag
25466,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS
25467,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
25468,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
25469,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS
25471,/home/app/src/data/shot-type-dataset/trailer_v...,Static,ECS
...,...,...,...
28840,/home/app/src/data/shot-type-dataset/trailer_v...,Push,FS
28871,/home/app/src/data/shot-type-dataset/trailer_v...,Push,LS
28894,/home/app/src/data/shot-type-dataset/trailer_v...,Push,ECS
28979,/home/app/src/data/shot-type-dataset/trailer_v...,Push,ECS


Renaming move_label as 'TAG'

In [133]:
train_df_bal = train_df_bal.rename(columns ={'tag':'scale_label'})
train_df_bal = train_df_bal.rename(columns ={'move_label':'tag'})
train_df_bal = train_df_bal.reset_index(drop = True)

test_df_bal = test_df_bal.rename(columns ={'tag':'scale_label'})
test_df_bal = test_df_bal.rename(columns ={'move_label':'tag'})
test_df_bal = test_df_bal.reset_index(drop = True)

In [41]:
#df_train.to_csv('/workspaces/final-project-shot-type/data/CSV/dataset_train_v1.csv')

In [42]:
#df_test.to_csv('/workspaces/final-project-shot-type/data/CSV/dataset_test_v1.csv')

In [134]:
train_df= train_df_bal
test_df = test_df_bal

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 2220
Total videos for testing: 380


,video_name,tag,scale_label
1319,/home/app/src/data/shot-type-dataset/trailer_v...,Pull,MS
1705,/home/app/src/data/shot-type-dataset/trailer_v...,Push,LS
200,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
588,/home/app/src/data/shot-type-dataset/trailer_v...,Static,FS
920,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,CS
1649,/home/app/src/data/shot-type-dataset/trailer_v...,Push,LS
396,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS
353,/home/app/src/data/shot-type-dataset/trailer_v...,Static,LS
193,/home/app/src/data/shot-type-dataset/trailer_v...,Static,ECS
581,/home/app/src/data/shot-type-dataset/trailer_v...,Static,FS


Checking correct video tagging with original JSON

In [155]:
# reading the JSON data using json.load()
json_path = '/home/app/src/data/shot-type-dataset/v1_split_trailer.json'

with open(json_path) as train_file:
    dict_v1= json.load(train_file)


def tag_ckecker (df, json_dict, df_name_to_check):
    tag_chek_list = []

    for i in range(len(df.index)):
        movie = df['video_name'][i]
        df_tag = df['tag'][df['video_name'] == movie][i]
        find_char_st = movie.find('_', 55,60)
        find_char_end = movie.find('_', 65,70)
        movie_key = movie[find_char_st+1:find_char_end]
        trailer_key =  movie[find_char_end+1:find_char_end+5]
        try:
            dict_tag = json_dict[df_name_to_check][movie_key][trailer_key]['movement']['label']
        except KeyError:
            dict_tag = json_dict['val'][movie_key][trailer_key]['movement']['label']

            if dict_tag == df_tag:
                pass
            else:
                tag_chek_list.append(movie)

    if len(tag_chek_list) == 0:
        print(f'tag check {df_name_to_check} OK')
    else: print (tag_chek_list)


tag_ckecker(train_df, dict_v1, 'train')
tag_ckecker(test_df, dict_v1, 'test')


tag check train OK
tag check test OK


In [158]:
train_df['tag'].value_counts()

Push      773
Static    600
Motion    600
Pull      247
Name: tag, dtype: int64

Define hyperparameters

In [160]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 20

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

Loading frames and working on frame's size

In [46]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

Feature extractor

In [47]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()
print(feature_extractor.summary())

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________
None


Working on labels

In [48]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['Motion', 'Pull', 'Push', 'Static']


Preparing videos

In [49]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Saving embeddings

In [50]:
path_save_embeddings= '/home/app/src/embeddings/mov_bal_2k_embeddings_20F/'

# Train
train_data_embedding_0 = np.save(path_save_embeddings + 'train_data_embedding_0.npy', train_data[0])
train_data_embedding_1 = np.save(path_save_embeddings + 'train_data_embedding_1.npy', train_data[1])
train_labels_embedding = np.save(path_save_embeddings + 'train_labels_embedding.npy', train_labels)

#Test
test_data_embedding_0 = np.save(path_save_embeddings + 'test_data_embedding_0.npy', test_data[0])
test_data_embedding_1 = np.save(path_save_embeddings + 'test_data_embedding_1.npy', test_data[1])
test_labels_embedding = np.save(path_save_embeddings + 'test_labels_embedding.npy', test_labels)

Loading embeddings

In [51]:
# path_load_embeddings= '/home/app/src/embeddings/mov_bal_2k_embeddings_20F/'

# #Train
# train_data_0 = np.load(path_load_embeddings + 'train_data_embedding_0.npy')
# train_data_1 = np.load(path_load_embeddings + 'train_data_embedding_1.npy')
# train_data=(train_data_0, train_data_1)
# train_labels = np.load(path_load_embeddings + 'train_labels_embedding.npy')

# #Test
# test_data_0 = np.load(path_load_embeddings + 'test_data_embedding_0.npy')
# test_data_1 = np.load(path_load_embeddings + 'test_data_embedding_1.npy')
# test_data=(test_data_0, test_data_1)
# test_labels = np.load(path_load_embeddings + 'test_labels_embedding.npy')

Sequence model (RNN)

In [161]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"],
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5"
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/20
49/49 [==============================] - ETA: 0s - loss: 1.2984 - accuracy: 0.3668
Epoch 1: val_loss improved from inf to 2.05898, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5
49/49 [==============================] - 13s 91ms/step - loss: 1.2984 - accuracy: 0.3668 - val_loss: 2.0590 - val_accuracy: 0.0000e+00
Epoch 2/20
49/49 [==============================] - ETA: 0s - loss: 1.2370 - accuracy: 0.3964
Epoch 2: val_loss did not improve from 2.05898
49/49 [==============================] - 2s 32ms/step - loss: 1.2370 - accuracy: 0.3964 - val_loss: 2.2430 - val_accuracy: 0.0000e+00
Epoch 3/20
49/49 [==============================] - ETA: 0s - loss: 1.2192 - accuracy: 0.4015
Epoch 3: val_loss did not improve from 2.05898
49/49 [==============================] - 2s 32ms/step - loss: 1.2192 - accuracy: 0.4015 - val_loss: 2.3507 - val_accuracy: 0.0000e+00
Epoch 4/20
49/49 [==============================] - ETA: 0s - loss: 1.2098 - accur

Predictions

In [53]:
# def prepare_single_video(frames):
#     frames = frames[None, ...]
#     frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#     frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

#     for i, batch in enumerate(frames):
#         video_length = batch.shape[0]
#         length = min(MAX_SEQ_LENGTH, video_length)
#         for j in range(length):
#             frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
#         frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#     return frame_features, frame_mask


# def sequence_prediction(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features, frame_mask = prepare_single_video(frames)
#     probabilities = sequence_model.predict([frame_features, frame_mask])[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

Test video path: /home/app/src/data/shot-type-dataset/trailer_v3/test/shot_tt2039393_0013.mp4


NameError: name 'sequence_prediction' is not defined